# Vector-Axes Implementation of Kao et al. 2014

### Importing packages

In [ ]:
import RSA.setup as s
from RSA.byRSAz import *

### Setting up configuration files & model

In [ ]:
#Imports validated data from Kao et al. 2014
dfde = s.pd.read_csv('data/kao-F-ANT.csv')

#Create a configuration class that generates relevant model inputs--
# U (source; s), F (intents, i), and A.
cf = s.config(sources='data/config/GLOVe/SOURCE.csv', intents='data/config/GLOVe/INTENTS.csv', axes='data/config/GLOVe/AXES.csv')
l = mod(cf)

### Validating feature extraction accuracy

In [ ]:
data = []
for row in dfde.values:
    sel_f = cf.sel_intent([row[-1]])
    
    try:
        vv = l.by_RSA.SNda(row[0], [row[3]])
        res = np.around(vv[0].view(-1)[sel_f].item(), 3)
        max_item = cf.intent([vv[0].view(-1).argmax().item()])[0]
        max_per = np.around(vv[0].view(-1).max().item(), 3)

        data += [[row[0], row[-1], res, max_item, max_per]]
    
    except IndexError:
        print(row[3])

df_report = pd.DataFrame(np.array(data), columns=['animal', 'f', 'P(f)', 'max.f', 'max%'])


### Model score

In [ ]:
df_report['TRUTH'] = df_report['f'] == df_report['max.f']
df_report['TRUTH'].mean()

### Validating how well the model can "pull out" a specific f|u

In [ ]:
U = cf.As.unsqueeze(1) * cf.Ai

#Validating the number of times that u is closer to f than other U
data = []
for row in dfde.values:
    vals = U[:,cf.sel_intent([row[-1]]),cf.sel_axis([row[3]])]
    u = vals[cf.sel_source([row[0]])].view(-1)
    resp = (u >= vals).float().mean()
    data.append([row[0], row[3], row[-1], np.around(resp.item(),3)])
data = pd.DataFrame(np.array(data).reshape(-1,4), columns=['animal', 'axis', 'f', '%>'])